In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd


path = "/content/drive/My Drive/fire_predict/base_data/"
df_train = pd.read_csv(path + "PJT002_train.csv")
df_val = pd.read_csv(path + "PJT002_validation.csv")
df_test = pd.read_csv(path + "PJT002_test.csv")

In [0]:
import math
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm


def f_data(df_train, df_val, df_test):

    X_train = df_train.drop(['fr_yn'], 1)
    y_train = df_train['fr_yn']
    X_val = df_val.drop(['fr_yn'], 1)
    y_val = df_val['fr_yn']
    X_test = df_test

    df_all = pd.concat([X_train, X_val, X_test])

    categorical_cols = df_all.select_dtypes(['object']).columns
    for col in categorical_cols:
        df_all[col] = pd.Categorical(df_all[col]).codes

    X_train = df_all[:len(df_train)]
    X_val = df_all[len(df_train):-len(df_test)]
    X_test = df_all[-len(df_test):]

    X_train = X_train.fillna(-1)
    X_val = X_val.fillna(-1)
    X_test = X_test.fillna(-1)

    return X_train, y_train, X_val, y_val, X_test

def f1_rfc(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

binary_y = {'N': 0, 'Y': 1}

df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

df_train_2 = df_train
df_val_2 = df_val
df_test_2 = df_test

df_train_2 = df_train_2.drop("id", axis=1)
df_val_2 = df_val_2.drop("id", axis=1)
df_test_2 = df_test_2.drop("id", axis=1)

df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
df_test_2 = df_test_2.drop("dt_of_fr", axis=1)


def year(x):
    if x != "NaN":
        if isinstance(x, float):
            if not math.isnan(x):
                if int(x) >10000000:
                    return x // 10000
                elif int(x) > 1000000:
                    return x // 1000
                elif int(x) > 100000:
                    return x // 100
                elif int(x) > 10000:
                    return x // 10
                elif int(x) > 1000:
                    return x
        elif isinstance(x, str):
            return int(x[:4])




df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

def population(x):
    if x < 1000:
        return "1k"
    elif x < 10000:
        return "10k"
    elif x < 100000:
        return "100k"
    elif x < 1000000:
        return "1000k"

df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
# f1_rfc(X_train, y_train, X_val, y_val)

In [0]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [0]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [0]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [0]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [0]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

In [0]:
random_search = RandomizedSearchCV(xgb, param_distributions=params,
                                   n_iter=param_comb, scoring='roc_auc',
                                   n_jobs=4, cv=skf.split(X_train,y_train), verbose=3,
                                   random_state=42 )

In [17]:
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed: 19.1min finished



 Time taken: 0 hours 21 minutes and 27.11 seconds.


In [18]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv(path + 'xgb-random-grid-search-results-01.csv', index=False)



 All results:
{'mean_fit_time': array([302.53391655, 283.48984973, 364.0247767 , 250.6140883 ,
       231.25773215]), 'std_fit_time': array([ 1.28197179,  1.0747841 ,  2.47153756,  0.4992266 , 21.43318056]), 'mean_score_time': array([1.46226621, 1.06107934, 1.00040611, 1.08984137, 0.78706408]), 'std_score_time': array([0.01946922, 0.02752614, 0.04567554, 0.03718603, 0.22233554]), 'param_subsample': masked_array(data=[0.8, 1.0, 0.8, 0.6, 0.6],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[10, 1, 10, 10, 10],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[4, 3, 3, 3, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[1.5, 2, 5, 1, 1],
             mask=[False, False, False, False, False],
       fill

In [0]:
folds = 3
param_comb = 50

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

In [0]:
random_search = RandomizedSearchCV(xgb, param_distributions=params,
                                   n_iter=param_comb, scoring='roc_auc',
                                   n_jobs=4, cv=skf.split(X_train,y_train), verbose=3,
                                   random_state=42 )

In [23]:
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 31.1min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 186.2min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed: 232.7min finished



 Time taken: 3 hours 55 minutes and 30.16 seconds.


In [24]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv(path + 'xgb-random-grid-search-results-01.csv', index=False)



 All results:
{'mean_fit_time': array([292.15210366, 273.73922245, 352.5693392 , 243.52383208,
       304.81564713, 330.78261447, 273.32416177, 327.62875239,
       448.31508795, 456.56517005, 436.77139656, 384.33283559,
       475.59905839, 500.84849938, 448.81349007, 408.57589324,
       407.77790316, 372.81470315, 322.31270115, 322.94872554,
       247.06261468, 583.53738109, 395.70632545, 394.66322009,
       549.74487011, 349.25864299, 257.70952598, 452.83143727,
       396.17990255, 323.36943372, 385.57435099, 308.74815234,
       258.04459707, 310.51201042, 309.0275689 , 393.43510167,
       230.28487722, 497.11750174, 249.21221073, 393.62958082,
       449.17117469, 540.88939691, 281.69876003, 249.26829926,
       342.57510066, 462.7371908 , 362.25083923, 436.45980748,
       358.66608445, 285.16990662]), 'std_fit_time': array([ 1.00382234,  0.71274098,  1.24515669,  0.50723806,  0.21479802,
        0.52974377,  0.54832179,  1.15718357,  1.33131632,  5.54636499,
        1.1518

In [0]:
params = {
        'min_child_weight': [10, 20, 30],
        'gamma': [5, 10, 15],
        'subsample': [0.1, 0.3, 0.5],
        'colsample_bytree': [0.1, 0.3, 0.5],
        'max_depth': [5, 10, 15]
        }

In [27]:
random_search = RandomizedSearchCV(xgb, param_distributions=params,
                                   n_iter=param_comb, scoring='roc_auc',
                                   n_jobs=4, cv=skf.split(X_train,y_train), verbose=3,
                                   random_state=42 )
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 34.9min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 141.6min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed: 168.3min finished



 Time taken: 2 hours 50 minutes and 52.89 seconds.


In [28]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv(path + 'xgb-random-grid-search-results-01.csv', index=False)



 All results:
{'mean_fit_time': array([ 91.44246554,  82.73309835, 350.2345252 , 434.49308983,
       540.63147918, 454.62720108, 621.98265791, 162.05238358,
       108.06776587, 200.18586707, 113.53766569, 748.3678391 ,
       543.36392633, 299.2595249 , 174.19515252, 381.32197046,
       735.22150787,  85.41433978,  95.55198296, 407.15581377,
        82.96044429, 180.70811478, 149.72391971,  85.86567156,
       743.68064086, 332.4560047 , 151.77604802, 182.56330323,
       203.25428605, 520.64443167, 150.85222856, 275.48965168,
       143.92272067,  82.82245851, 124.07827497, 264.75796787,
       178.31565293, 278.73058303, 225.57908106, 258.06999509,
       105.66095455, 243.66267014, 191.8521661 , 138.32199295,
       210.12431701, 143.05382363, 330.23080381, 180.67380579,
       394.27311595, 173.61896578]), 'std_fit_time': array([ 0.24787215,  0.40819355,  1.42273587,  1.38605218,  4.09970502,
        3.82760409,  3.78372377,  0.73554222,  0.4315879 ,  1.1181959 ,
        0.3801

In [0]:
params = {
        'min_child_weight': [6, 7, 8],
        'gamma': [3, 4, 5],
        'subsample': [0.5, 0.55, 0.6],
        'colsample_bytree': [0.5, 0.55, 0.6],
        'max_depth': [4, 5, 6]
        }

In [15]:
random_search = RandomizedSearchCV(xgb, param_distributions=params,
                                   n_iter=param_comb, scoring='roc_auc',
                                   n_jobs=4, cv=skf.split(X_train,y_train), verbose=3,
                                   random_state=42 )
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 37.1min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 172.4min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed: 211.0min finished



 Time taken: 3 hours 33 minutes and 44.78 seconds.


In [16]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv(path + 'xgb-random-grid-search-results-01.csv', index=False)



 All results:
{'mean_fit_time': array([352.77071834, 258.06279039, 333.37327154, 358.82642857,
       418.8762211 , 360.67398206, 418.80987755, 335.16817435,
       310.27042079, 282.87788852, 359.10493787, 421.25580692,
       362.52334054, 303.3789111 , 278.74354251, 384.99767836,
       417.35165135, 307.69629192, 309.54323562, 390.40638836,
       261.61129514, 364.99679653, 278.51759283, 262.5865763 ,
       417.56991839, 301.98233151, 316.26088667, 364.96297908,
       333.38592847, 396.18243257, 281.67781965, 308.48003292,
       371.39320238, 262.16168833, 364.442041  , 390.10191878,
       330.20181243, 302.29574656, 283.08281938, 408.99475034,
       361.84554394, 330.91589189, 282.84003854, 315.49622655,
       295.40529394, 315.16294456, 302.40476441, 313.03007404,
       406.82696342, 227.15775895]), 'std_fit_time': array([ 0.34341004,  0.725045  ,  1.22415234,  0.78364045,  2.0612225 ,
        2.93721051,  0.93087315,  1.07870378,  1.6848652 ,  1.85441988,
        4.1995

In [0]:
# grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train, y_train), verbose=3 )
# grid.fit(X_train, y_train)
# print('\n All results:')
# print(grid.cv_results_)
# print('\n Best estimator:')
# print(grid.best_estimator_)
# print('\n Best score:')
# print(grid.best_score_ * 2 - 1)
# print('\n Best parameters:')
# print(grid.best_params_)
# results = pd.DataFrame(grid.cv_results_)
# results.to_csv(path + 'xgb-grid-search-results-01.csv', index=False)

In [0]:
params = {
        'min_child_weight': [6, 7, 8],
        'gamma': [3, 4, 5],
        'subsample': [0.5, 0.55, 0.6],
        'colsample_bytree': [0.5, 0.55, 0.6],
        'max_depth': [4, 5, 6]
        }

In [18]:
random_search = RandomizedSearchCV(xgb, param_distributions=params,
                                   n_iter=param_comb, scoring='roc_auc',
                                   n_jobs=4, cv=skf.split(X_train,y_train), verbose=3,
                                   random_state=42 )
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 38.8min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 175.5min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed: 213.9min finished



 Time taken: 3 hours 36 minutes and 37.88 seconds.


In [19]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv(path + 'xgb-random-grid-search-results-01.csv', index=False)



 All results:
{'mean_fit_time': array([370.54001355, 268.39077846, 345.33716718, 371.32850877,
       430.38563983, 370.06346965, 434.40789239, 343.06200957,
       318.69172748, 289.62692285, 370.81054266, 430.6811525 ,
       373.10454567, 309.74867312, 290.0206542 , 398.18560576,
       431.25420284, 317.72874848, 315.80497956, 397.71961848,
       266.00624593, 372.74092269, 284.87339846, 266.1085341 ,
       426.59720945, 306.83073791, 317.50157809, 366.13507438,
       334.54609505, 391.90364408, 279.77430161, 302.48154434,
       364.20865583, 259.63674394, 359.50791272, 383.39090482,
       330.38262979, 301.48052748, 282.83902582, 413.28387054,
       360.22564046, 333.20631123, 283.49287669, 315.38083744,
       299.70181807, 315.87299911, 303.93024445, 315.71521584,
       409.02067828, 224.3875014 ]), 'std_fit_time': array([ 2.30802392,  3.63229806,  1.91668323,  4.30694034,  4.47494142,
        2.26941471,  5.61447585,  5.87683911,  2.37088097,  2.71873187,
        4.1770